To run first place Posts_Coffee.xml, and post_parser_record.py in session storage, then create a folder in session storage named Entities and place Post.py inside, then run code sections in order

In [ ]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")
!pip3 install nltk
import nltk
nltk.download('punkt')
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Creates Inverted Index

In [ ]:
#This section is to prepare the dictionary dic, and use tokens to assign each 
#document id with the ammount of occurences to a specific token
#example dic will contain an inverted index such as {"espresso": {1: 4}, {2: 1}}  
#if espresso in document 1 had 4 matches, and 1 match in document 2

stop_words = stopwords.words('english')
#print(stopwords.words('english'))

#initializes variables 
recall = []
precision = []
matches_found = 0;
i= 0
values = {}
dic = {}

#discovered how to use functions in python!
def tokenize(tokens, id, dic):
  #goes through each token 
  for word in tokens: 
    #stopwords shall not pass
    if word not in stop_words:
      #if not in dic, it creates a new entry and gives it one occurrence
      if word not in dic: 
        dic[word] = {id:1}
      else:
        #else if word exists in dic, but id does not, creates a new entry and gives it one occurrence
        if id not in dic[word]: 
          dic[word][id] = 1
        else:
          #else the token and id allready exists in dic, so occurrence++
          dic[word][id] += 1

#runs through every question post
for question_id in post_reader.map_questions:
  #tokenizes for questions
  question = post_reader.map_questions[question_id]
  tokens = word_tokenize(re.sub(r'[^a-z]', ' ', question.title.lower()) + re.sub(r'[^a-z]', ' ', question.body.lower()))
  tokenize(tokens, question_id, dic)

#runs through every answer post
for answer_id in post_reader.map_just_answers:
  #tokenizes for questions
  answer = post_reader.map_just_answers[answer_id]
  tokens = word_tokenize(re.sub(r'[^a-z]', ' ', answer.body.lower()))
  tokenize(tokens, answer_id, dic)



In [ ]:
#run to simply initalize this function for use 
def search(keys,words,query_num):
  temp_dic = {}
  
  seperated_words = words.split(" ")
  #checks that the query is larger than one word
  if len(seperated_words) is not 1:
    #ensures full query is valid
    for word in seperated_words:  
      if word not in dic:
        return ""
      #scans through each key 
      for current_key in keys:
        bad_match = False
        #checks if key is held for all words of query
################################################################################
        for word in seperated_words:
          if int(str(current_key)) not in dic[word]:
            bad_match = True
        if(bad_match == False):
          temp_val = 0
          for word in seperated_words:
            temp_val += dic[word][current_key]
          temp_dic[current_key] = temp_val
################################################################################
  #query is only one word
  else:
    for current_key in keys: 
      #simply copys the dictionary of specific word to temp_dic
      temp_dic = dic.copy()[words]
  query_num+=1 

  #reverse sorts, by largest amount of occurrence first
  myList = sorted(temp_dic.items(), key=lambda x: x[1], reverse=True)
  i = 0
  for items in myList:
    #makes sure only 10 of each pass are printed
    if i <= 9:
      print(str(items[0])+"    "+str(items[1]))
      i += 1
    else:
      break

In [ ]:
#Querys
#simply add Query to list 
querys = ["espresso", "turkish coffee","persian coffee"]
keys = []
query_num = 0

#scans through and prints each query
for words in querys:
  print(words)
  #seperates query into a list
  seperated_query = words.split(" ")
  #scans through each word in query specifically to save all matching occurances to the list of keys
  for query_part in seperated_query:
    if query_part in dic:
      #adds keys of matching documents for current word in query
      keys = keys + list(dic[query_part].keys())
      #sorts keys to keep organization for my sanity
  keys.sort()
  ordered_keys = []
  #removes duplicate keys
  [ordered_keys.append(x) for x in keys if x not in ordered_keys]
  #uses function to index and print final result
  search(ordered_keys,words,query_num)
  query_num += 1
  keys = []
  



espresso
3269    34
1574    27
2095    22
283    17
2077    14
5537    14
1651    13
2087    12
2116    12
3721    12
turkish coffee
2392    70
3710    30
4407    22
3101    20
4185    20
1545    19
1833    19
4273    19
165    18
2394    16
persian coffee


espresso

ID:      Precision

3269:    1

1574:    1

2095:    1

283:     .75

2077:    .8

5537:    .83

1651:    .85

2087:    .87

2116:    .88

3721:    .9

turkish coffee

ID: Precision

2392:    1

3710:    .5

4407:   .66

3101:    .5

4185:    .4

1545:    .33

1833:    .42

4273:    .37

165:     .33

2394:    .3

persian coffee

N/A








